In [21]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/scripts-code/text/Whisper_Evaluation_small.ipynb
/kaggle/input/scripts-code/text/load_ground_truth_common_voice.py
/kaggle/input/scripts-code/text/definitions.py
/kaggle/input/scripts-code/text/wazemour@bocal.cs.univ-paris8.fr
/kaggle/input/scripts-code/text/Evaluation_Departement_Banque.ipynb
/kaggle/input/scripts-code/text/transcriber_whisper.py
/kaggle/input/scripts-code/text/Whisper_Evaluation_tiny_2000.ipynb
/kaggle/input/scripts-code/text/normalizer.py
/kaggle/input/scripts-code/text/Whisper_Evaluation_base_2000.ipynb
/kaggle/input/scripts-code/text/filter_char.py
/kaggle/input/scripts-code/text/transcriber.py
/kaggle/input/scripts-code/text/gateway-bocal
/kaggle/input/scripts-code/text/utils.py
/kaggle/input/scripts-code/text/MP3toWAV.py
/kaggle/input/scripts-code/text/tokenizer.py
/kaggle/input/scripts-code/text/load_ground_truth.py
/kaggle/input/scripts-code/text/Whisper_Evaluation_tiny.ipynb
/kaggle/input/scripts-code/text/evaluate_wer_cer.py
/kaggle/input/scrip

In [22]:
!pip install -q transformers datasets torchaudio

In [23]:
!pip install -q librosa soundfile

In [24]:
from transformers import pipeline

In [25]:
asr = pipeline("automatic-speech-recognition", model="gweltou/wav2vec2-xls-r-300m-br")

Device set to use cuda:0


In [74]:
import sys

sys.path.append('/kaggle/input/scripts-code/text')

import os
import wave
import json
import audio


In [28]:
!pip install jiwer

In [75]:
import jiwer

In [76]:
import pandas as pd
df = pd.read_parquet("hf://datasets/Bretagne/Banque_Sonore_Dialectes_Bretons/train.parquet")

In [77]:
def fix_row(row):
    if 'peurunvan :' in row['fr']:
        # Extract Breton text after "peurunvan :"
        new_text = row['fr'].split('peurunvan :', 1)[1].strip()
        # Replace both Br and Fr columns with this new_text
        row['br'] = new_text
        row['fr'] = new_text
    return row

df = df.apply(fix_row, axis=1)

In [78]:
df

,audio,br,fr,city
0,{'bytes': b'RIFF$x\x06\x00WAVEfmt \x10\x00\x00...,Pell zo n'eo ket o welet ac'hanon.,Ça fait longtemps qu'il n'est pas venu me voir.,22107
1,{'bytes': b'RIFF$\xb1\x03\x00WAVEfmt \x10\x00\...,N'oc'h ket bet pell zo o welet ac'hanon.,Vous n'êtes pas venus me voir depuis longtemps.,22107
2,{'bytes': b'RIFF$r\x03\x00WAVEfmt \x10\x00\x00...,Pell zo n'eo ket bet o welet ac'hanon.,Ça fait longtemps qu'il n'est pas venu me voir.,22107
3,{'bytes': b'RIFF$\x97\x05\x00WAVEfmt \x10\x00\...,Ma... ma zi ne vo ket gwerzhet.,Ma... ma maison ne sera pas vendue.,22107
4,{'bytes': b'RIFF$\xa0\x05\x00WAVEfmt \x10\x00\...,Hon... hon ti ne vo ket gwerzhet.,Notre... notre maison ne sera pas vendue.,22107
...,...,...,...,...
7286,{'bytes': b'RIFF$\xce\x07\x00WAVEfmt \x10\x00\...,Ne zeue ket ag ar vro Kemper. Ne zeue ket ag a...,Il ne venait pas du pays de Quimper. Il ne ven...,56247
7287,{'bytes': b'RIFF$\xa6\x08\x00WAVEfmt \x10\x00\...,Bout a oa linad razh d'en-dro ar feunteun. Bou...,Il y avait des orties tout autour de la fontai...,56247
7288,{'bytes': b'RIFF$i\x03\x00WAVEfmt \x10\x00\x00...,Ne oa ket aes o zennañ.,Ce n'était pas facile de les arracher.,56247
7289,{'bytes': b'RIFF$3\x03\x00WAVEfmt \x10\x00\x00...,Ne oa ket aes o zennañ.,Ce n'était pas facile de les arracher.,56247


In [81]:
small_df = df.sample(n=2000, random_state=42)  # random_state makes it reproducible
small_df

,audio,br,fr,city
1057,{'bytes': b'RIFF\x8c\xdf\x00\x00WAVEfmt \x10\x...,ur c'harzh,une haie,29032
2654,{'bytes': b'RIFF\xae\xe5\x04\x00WAVEfmt \x10\x...,Mari zo deuet a-benn enno antronoz.,Marie est venue vers eux le lendemain.,29168
4401,{'bytes': b'RIFF$1\x08\x00WAVEfmt \x10\x00\x00...,"An amzer gwechall, ne oa... ne oa ket, eu... n...","Autrefois, c'était... c'était, euh... ça n'éta...",29230
5843,{'bytes': b'RIFF\xa4\x81\x03\x00WAVEfmt \x10\x...,Piv zo bet o troc'hiñ ar wezenn ?,Qui a coupé l'arbre ?,29293
7177,{'bytes': b'RIFF$e\x04\x00WAVEfmt \x10\x00\x00...,"Pa oufemp ar wirionez, n'he lavarfemp ket.","Si nous savions la vérité, nous ne la dirions ...",56175
...,...,...,...,...
981,{'bytes': b'RIFF8@\x01\x00WAVEfmt \x10\x00\x00...,skarzhañ,vider,29032
1732,{'bytes': b'RIFF$\xc4\x05\x00WAVEfmt \x10\x00\...,Lak da zornoù e-barzh da boch ! Tenn anezhe er...,Mets tes mains dans ta poche ! Retire-les d'en...,29150
6185,{'bytes': b'RIFF$F\x05\x00WAVEfmt \x10\x00\x00...,"Nend, nend ay ket ganin bremañ, c'hwi [?], me ...","Nend, nend ay ket ganin bremañ, c'hwi [?], me ...",56076
2354,{'bytes': b'RIFF\x08j\x06\x00WAVEfmt \x10\x00\...,"Oh nann, mes me a gred lar mar vije bet chomet...","Oh nann, mes me a gred lar mar vije bet chomet...",29153


In [79]:
dataset_path = "/kaggle/input/banque-sonore-data/Audio"

In [80]:
from load_ground_truth import load_ground_truth_dict

In [91]:
def generate_transcription(row):
    file_id = row['file']
    utt_name = f"utt_{file_id:05d}.wav"
    full_path = os.path.join(dataset_path, utt_name)
    
    result = asr(full_path)
    transcription = result['text']
    
    print(f"{utt_name} → {transcription}")
    
    return transcription


In [83]:
result = asr("/kaggle/input/banque-sonore-data/Audio/utt_00000.wav")
print(result["text"])

pell zo n'eo ket bet  e c'hwel'h anañ


In [92]:
small_df['file'] = small_df.index

small_df['transcription'] = small_df.apply(generate_transcription, axis=1)

# Optional: Reset index for a clean new DataFrame
small_df = small_df.reset_index(drop=True)

utt_01057.wav → c'hazh
utt_02654.wav → mori a zo deuet benn nee internoz
utt_04401.wav → un amzour gwechel oa oa ket euh oa tu memes mod
utt_05843.wav → piv zo bet he troewin
utt_07177.wav → ha poem ar virioni ni lavaremp ket
utt_03805.wav → heskert choentoù nin ba sur
utt_03961.wav → pent'm' eus labourat'm eus goneumet se
utt_06147.wav → nam zo bet  doua fetenn rei eo va prag
utt_03551.wav → a wechoù m' eus poant benn da c'hda c'hell pad deizh
utt_02413.wav → greich lout
utt_00185.wav → a vesur  koaz ta mic'hañ a sperad dezhañ
utt_01221.wav → ha hag glesk a oa bet kub ematre bar  c'holi n'n'euz ket bro añ ma toul duban bezmañ ambous e vezmañ ar gleskauedn a neuze vras fouget eo torztu ar c'hi ar c'hizout  ar c'hi en'eus kenneer vez goll lashiral ya atre ar c'hi n'eus ket bent i ta n'e c'houlz ket
utt_00061.wav → vez gouel an holl senn a blez tro menet c'hwac'h muioc'h ar borkdoùu e e vert hwit er blez mañ
utt_02328.wav → te zo be tuskont dibrit koihet bar bro mok oiet
utt_06524.wav → 

In [95]:
small_df

,audio,br,fr,city,file,transcription
0,{'bytes': b'RIFF\x8c\xdf\x00\x00WAVEfmt \x10\x...,ur c'harzh,une haie,29032,1057,c'hazh
1,{'bytes': b'RIFF\xae\xe5\x04\x00WAVEfmt \x10\x...,Mari zo deuet a-benn enno antronoz.,Marie est venue vers eux le lendemain.,29168,2654,mori a zo deuet benn nee internoz
2,{'bytes': b'RIFF$1\x08\x00WAVEfmt \x10\x00\x00...,"An amzer gwechall, ne oa... ne oa ket, eu... n...","Autrefois, c'était... c'était, euh... ça n'éta...",29230,4401,un amzour gwechel oa oa ket euh oa tu memes mod
3,{'bytes': b'RIFF\xa4\x81\x03\x00WAVEfmt \x10\x...,Piv zo bet o troc'hiñ ar wezenn ?,Qui a coupé l'arbre ?,29293,5843,piv zo bet he troewin
4,{'bytes': b'RIFF$e\x04\x00WAVEfmt \x10\x00\x00...,"Pa oufemp ar wirionez, n'he lavarfemp ket.","Si nous savions la vérité, nous ne la dirions ...",56175,7177,ha poem ar virioni ni lavaremp ket
...,...,...,...,...,...,...
1995,{'bytes': b'RIFF8@\x01\x00WAVEfmt \x10\x00\x00...,skarzhañ,vider,29032,981,diskerzh eo
1996,{'bytes': b'RIFF$\xc4\x05\x00WAVEfmt \x10\x00\...,Lak da zornoù e-barzh da boch ! Tenn anezhe er...,Mets tes mains dans ta poche ! Retire-les d'en...,29150,1732,laka da zor en obad ta boch tenn nemes joust a...
1997,{'bytes': b'RIFF$F\x05\x00WAVEfmt \x10\x00\x00...,"Nend, nend ay ket ganin bremañ, c'hwi [?], me ...","Nend, nend ay ket ganin bremañ, c'hwi [?], me ...",56076,6185,h dei ket gan embre ba vi n oant memup tichañs...
1998,{'bytes': b'RIFF\x08j\x06\x00WAVEfmt \x10\x00\...,"Oh nann, mes me a gred lar mar vije bet chomet...","Oh nann, mes me a gred lar mar vije bet chomet...",29153,2354,onan ve me ga la' ma vez bet chames fe bet dis...


In [96]:
from filter_char import filter_out_chars
from normalizer import normalize_sentence
from utils import pre_process


def process_br_text(br):
    PUNCTUATION = '<>.?!,;:«»“”"()[]/…–—•'
    br = filter_out_chars(br, PUNCTUATION + '*')
    br = normalize_sentence(br, autocorrect=True)
    br = pre_process(br).replace('-', ' ').lower()
    return br

In [97]:
small_df['br_processed'] = small_df['br'].apply(process_br_text)

In [98]:
from jiwer import wer, cer

# Compute WER and CER for each row
small_df['wer'] = small_df.apply(lambda row: wer(row['br_processed'], row['transcription']), axis=1)
small_df['cer'] = small_df.apply(lambda row: cer(row['br_processed'], row['transcription']), axis=1)

In [99]:
small_df = small_df[['br', 'br_processed', 'fr', 'transcription', 'city', 'wer', 'cer', 'file']]
small_df

,br,br_processed,fr,transcription,city,wer,cer,file
0,ur c'harzh,ur c'harzh,une haie,c'hazh,29032,1.000000,0.400000,1057
1,Mari zo deuet a-benn enno antronoz.,mari zo deuet a benn enno antronoz,Marie est venue vers eux le lendemain.,mori a zo deuet benn nee internoz,29168,0.714286,0.323529,2654
2,"An amzer gwechall, ne oa... ne oa ket, eu... n...",an amzer gwechall ne oa ne oa ket eu ne oa ket...,"Autrefois, c'était... c'était, euh... ça n'éta...",un amzour gwechel oa oa ket euh oa tu memes mod,29230,0.571429,0.303571,4401
3,Piv zo bet o troc'hiñ ar wezenn ?,piv zo bet o troc'hiñ ar wezenn,Qui a coupé l'arbre ?,piv zo bet he troewin,29293,0.571429,0.483871,5843
4,"Pa oufemp ar wirionez, n'he lavarfemp ket.",pa oufemp ar wirionez n'he lavarfemp ket,"Si nous savions la vérité, nous ne la dirions ...",ha poem ar virioni ni lavaremp ket,56175,0.714286,0.300000,7177
...,...,...,...,...,...,...,...,...
1995,skarzhañ,skarzhañ,vider,diskerzh eo,29032,2.000000,0.750000,981
1996,Lak da zornoù e-barzh da boch ! Tenn anezhe er...,lak da zornoù e barzh da boch tenn anezhe er m...,Mets tes mains dans ta poche ! Retire-les d'en...,laka da zor en obad ta boch tenn nemes joust a...,29150,0.785714,0.424242,1732
1997,"Nend, nend ay ket ganin bremañ, c'hwi [?], me ...",nend nend ay ket ganin bremañ c'hwi me am eus ...,"Nend, nend ay ket ganin bremañ, c'hwi [?], me ...",h dei ket gan embre ba vi n oant memup tichañs...,56076,0.937500,0.536585,6185
1998,"Oh nann, mes me a gred lar mar vije bet chomet...",oh nann mes me a gred lâr mar vije bet chomet ...,"Oh nann, mes me a gred lar mar vije bet chomet...",onan ve me ga la' ma vez bet chames fe bet dis...,29153,0.850000,0.465909,2354


In [59]:
df['sentence_processed'] = df['sentence'].apply(process_br_text)

In [61]:
from jiwer import wer, cer

# Compute WER and CER for each row
df['wer'] = df.apply(lambda row: wer(row['sentence_processed'], row['Transcription']), axis=1)
df['cer'] = df.apply(lambda row: cer(row['sentence_processed'], row['Transcription']), axis=1)

In [63]:
df = df[['sentence', 'sentence_processed', 'Transcription', 'wer', 'cer','gender','sentence_domain','age','accents']]
df

,sentence,sentence_processed,Transcription,wer,cer,gender,sentence_domain,age,accents
0,Kanañ a raent en hent don.,kanañ a raent en hent don,kanañ a raint en en don,0.333333,0.120000,NaN,general,NaN,NaN
1,digor e vez da Verc'her.,digor e vez da verc'her,digor vez da vec'her,0.400000,0.130435,NaN,NaN,NaN,NaN
2,Re nebeut a zo deuet da reiñ dorn dimp.,re nebeut a zo deuet da reiñ dorn deomp,re nebeut a zo dibet da rein d'an deomper,0.444444,0.179487,NaN,NaN,NaN,NaN
3,Kemerit ho tafar hag azezit !,kemerit ho tafar hag azezit,kemerit ho tafoc'h hag azezit,0.200000,0.148148,NaN,NaN,NaN,NaN
4,Be'h dezhi !,be'h dezhi,bec'hchwa davoued,1.000000,1.100000,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...
2860,Ar memes tra a vo gulennet diganeoc'h.,ar memes tra a vo goulennet diganeoc'h,ar memes tra a vo goulennet diganeoc'h,0.000000,0.000000,NaN,NaN,NaN,NaN
2861,Alies em galv da zont da reiñ dorn dezhañ.,alies em galv da zont da reiñ dorn dezhañ,alies em galv da zont da reiñ dorn dezhañ,0.000000,0.000000,NaN,NaN,NaN,NaN
2862,Me/ Te. Eñ/Hi.,me te eñhi,me te eñhi,0.000000,0.000000,NaN,NaN,NaN,NaN
2863,Ar c'harr-tan ruz nevez-flamm-hont.,ar c'harr tan ruz nevez flamm hont,ar c'hartan ruz nevez flamm maon,0.428571,0.147059,NaN,NaN,NaN,NaN


In [100]:
city_avg = small_df.groupby('city')[['wer', 'cer']].mean()

# Optional: round to 2 decimal places
city_avg = city_avg.round(2)

for city_code, row in city_avg.iterrows():
    print(f"Average WER and CER for {city_code}:\nWER: {row['wer']}\nCER: {row['cer']}\n")


Average WER and CER for 22107:
WER: 0.77
CER: 0.38

Average WER and CER for 22167:
WER: 0.79
CER: 0.38

Average WER and CER for 22181:
WER: 0.94
CER: 0.48

Average WER and CER for 22230:
WER: 0.7
CER: 0.32

Average WER and CER for 22331:
WER: 0.73
CER: 0.34

Average WER and CER for 22336:
WER: 0.63
CER: 0.27

Average WER and CER for 22386:
WER: 0.82
CER: 0.37

Average WER and CER for 29005:
WER: 0.85
CER: 0.47

Average WER and CER for 29031:
WER: 1.85
CER: 1.06

Average WER and CER for 29032:
WER: 0.82
CER: 0.44

Average WER and CER for 29058:
WER: 0.8
CER: 0.41

Average WER and CER for 29080:
WER: 0.66
CER: 0.22

Average WER and CER for 29082:
WER: 0.9
CER: 0.42

Average WER and CER for 29091:
WER: 0.73
CER: 0.31

Average WER and CER for 29105:
WER: 0.47
CER: 0.18

Average WER and CER for 29122:
WER: 0.82
CER: 0.45

Average WER and CER for 29136:
WER: 0.8
CER: 0.45

Average WER and CER for 29139:
WER: 0.95
CER: 0.45

Average WER and CER for 29146:
WER: 0.72
CER: 0.36

Average WER and 

In [101]:
# Filter for cities starting with '22'
small_df_22 = small_df[small_df['city'].str.startswith('22')]
avg_22 = small_df_22[['wer', 'cer']].mean().round(4)


# Filter for cities starting with '22'
small_df_29 = small_df[small_df['city'].str.startswith('29')]
avg_29 = small_df_29[['wer', 'cer']].mean().round(4)


# Filter for cities starting with '56'
small_df_56 = small_df[small_df['city'].str.startswith('56')]
avg_56 = small_df_56[['wer', 'cer']].mean().round(4)



print("Evaluation for --Côtes-d'Armor-- dialects 22xxx :")
print(f"WER: {avg_22['wer']}")
print(f"CER: {avg_22['cer']}\n")

print("Evaluation for --Finistère-- dialects 29xxx :")
print(f"WER: {avg_29['wer']}")
print(f"CER: {avg_29['cer']}\n")

print("Evaluation for --Morbihan-- dialects 56xxx :")
print(f"WER: {avg_56['wer']}")
print(f"CER: {avg_56['cer']}")

Evaluation for --Côtes-d'Armor-- dialects 22xxx :
WER: 0.7435
CER: 0.3504

Evaluation for --Finistère-- dialects 29xxx :
WER: 0.8214
CER: 0.4201

Evaluation for --Morbihan-- dialects 56xxx :
WER: 0.8787
CER: 0.4627


In [102]:
average_wer = small_df['wer'].mean()
average_cer = small_df['cer'].mean()

# Optional: round them
average_wer = round(average_wer, 2)
average_cer = round(average_cer, 2)

print(f"Overall Average WER: {average_wer}")
print(f"Overall Average CER: {average_cer}")

Overall Average WER: 0.82
Overall Average CER: 0.42
